# Create crowd input workflow


1.) Create all questions of a run

- this should not be changed (only created once)
- if the question formulations change, we create a new run
- identifiers should not change
- create unique ids here
- we draw from this selection for creating batches 


2.) Create batch

- draw from all questions of a run
- only select properties in the experiment group
- always make sure that a batch contains the full set of questions of a pair 
- add prolific url 
- check prolific_input for already posed pairs
- write to file without header


In [ ]:
# match question ids - done
# to clean up the data, change the ids in the questions file to the input ids - done


# then check how much has been annotated

